# Persian Stance Classification - Deep Learning

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import numpy as np
import numpy as np
import os.path as path

# Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# input files
cleaned_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/Clean_Claim_Body.csv"
train_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/train_data.csv"
test_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/test_data.csv"

# Read Cleaned Data from CSV File

In [ ]:
import pandas as pd
dataset_clean = pd.read_csv(cleaned_path, index_col = 0, )

In [ ]:
clean_claim = dataset_clean['claim']
clean_body = dataset_clean['body']

In [ ]:
dataset_clean.head()

,claim,body,label
0,کلاهبرداری از رانندگان با شگرد نشت بنزین !,به گزارش خبرنگار گروه جامعه خبرگزاری میزان،29 ...,Discuss
1,تجاوز به دختر بازداشت شده و واژگونی ون گشت ارش...,انتشار کلیپ واژگونی ماشین گشت ارشاد توسط مردم ...,Discuss
2,تعظیم 20 دقیقه ای وزیر نیرو ژاپن به علت قطع بر...,وزیر نیروی ژاپن به علت قطع شدن برق؛ به همان مد...,Agree
3,سرمربیگری گاس هیدینک برای تراکتورسازی,به تازگی محمد تقوی استعفای خود را از سرمربیگری...,Discuss
4,کشف موجود عجیبی شبیه انسان در یک حفاری در پاکس...,پس از 20 سال حفاری با دقتی باورنکردنی، سرانجام...,Unrelated


# Read Data Train and Test from CSV Files

In [ ]:
data_train = pd.read_csv(train_path, index_col = 0, )
data_test = pd.read_csv(test_path, index_col = 0, )

In [ ]:
len(data_train), len(data_test)

(1597, 400)

# Base line (Majority Voting)

majority vote

In [ ]:
X_train = data_train['claim']
X_test = data_test['claim']

y_train = data_train['label']
y_test = data_test['label']

In [ ]:
%%time

from sklearn.dummy import DummyClassifier

clf_maj = DummyClassifier(strategy="most_frequent")

clf_maj.fit(X_train, y_train)

y_pred_maj = clf_maj.predict(X_test)

CPU times: user 359 ms, sys: 71.7 ms, total: 431 ms
Wall time: 405 ms


In [ ]:
LABELS = ['Agree', 'Disagree', 'Discuss', 'Unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm

In [ ]:
score, cm = score_submission(y_test, y_pred_maj)
fold_score, _ = score_submission(y_test, y_pred_maj)
max_fold_score, _ = score_submission(y_test, y_test)
score = fold_score / max_fold_score

print("FNC-1 score from restored model: " +  str(score) +"\n")

FNC-1 score from restored model: 0.5799180327868853



In [ ]:
def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))

In [ ]:
print_confusion_matrix(cm)

-------------------------------------------------------------
|           |   Agree   | Disagree  |  Discuss  | Unrelated |
-------------------------------------------------------------
|   Agree   |     0     |     0     |    27     |     0     |
-------------------------------------------------------------
| Disagree  |     0     |     0     |    42     |     0     |
-------------------------------------------------------------
|  Discuss  |     0     |     0     |    195    |     0     |
-------------------------------------------------------------
| Unrelated |     0     |     0     |    136    |     0     |
-------------------------------------------------------------


In [ ]:
from sklearn.metrics import classification_report

print( classification_report(y_test, y_pred_maj) )

              precision    recall  f1-score   support

       Agree       0.00      0.00      0.00        27
    Disagree       0.00      0.00      0.00        42
     Discuss       0.49      1.00      0.66       195
   Unrelated       0.00      0.00      0.00       136

    accuracy                           0.49       400
   macro avg       0.12      0.25      0.16       400
weighted avg       0.24      0.49      0.32       400



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
